In [1]:
# Step 1: Install dependencies
!pip install transformers accelerate -q
!pip install torch torchvision -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
# Step 2: Import libraries
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch


2025-05-12 20:28:26.820375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747081707.057515      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747081707.122036      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Step 3: Load model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16,
    device_map="auto"  # Automatically uses GPU if available
)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [4]:
generation_args = {
    "max_new_tokens": 3,          # Strict upper limit (allows short answers)
    "min_new_tokens": 1,           # Force at least 1 token
    "num_beams": 6,                # Beam search width
    "early_stopping": True,        # Stop early when confident
    "repetition_penalty": 1.2,     # Discourage repeated words
    "length_penalty": -1.0,        # Negative = favor shorter answers
    "no_repeat_ngram_size": 1,     # Prevent 2+ word repetitions
    "do_sample": False,           # Explicitly state beam search mode
    "num_return_sequences": 1     # Only return best answer
}

In [5]:
# 🔍 Test with one image
image_path = "/kaggle/input/abo-small-imgs/abo-images-small/abo-images-small/00/000b3f52.jpg"
question = "Answer in a single word: What is the case material?"

image = Image.open(image_path).convert("RGB")
inputs = processor(images=image, text=question, return_tensors="pt").to(device, torch.float16)

out = model.generate(**inputs,**generation_args)      # only one answer
answer = processor.decode(out[0], skip_special_tokens=True)

print(f"Q: {question}\nA: {answer}")

The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Q: Answer in a single word: What is the case material?
A: plastic


In [6]:
import pandas as pd
import re

# Load your CSV
df = pd.read_csv("/kaggle/input/abo-small-imgs/proper_result_2_final.csv")

# Add constraint to each question
df['question'] = df['question'].apply(lambda q: f"Answer in a ONE WORD ONLY:  {q.strip()}")

# Save back or to a new CSV
df.to_csv("test_1_oneword.csv", index=False)
print("Updated CSV saved as test_1_oneword.csv with 'Answer in one word' prompt.")


Updated CSV saved as test_1_oneword.csv with 'Answer in one word' prompt.


In [7]:
df.shape

(59847, 6)

In [8]:
print(f"Model device: {next(model.parameters()).device}")

Model device: cuda:0


In [9]:
import os
import sys
import re
import pandas as pd
from PIL import Image
from tqdm import tqdm
import contextlib
from transformers import logging as hf_logging

# Silence Hugging Face logging
hf_logging.set_verbosity_error()

# Suppress stdout and stderr (e.g. warnings from accelerate or transformers)
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = devnull, devnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

# Load dataset
df = pd.read_csv("test_1_oneword.csv")
batch_size = 2000
total_rows =len(df)
num_batches = (total_rows + batch_size - 1) // batch_size

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_rows)
    subset_df = df.iloc[start_idx:end_idx].copy()
    results = []

    for idx, row in tqdm(subset_df.iterrows(), total=len(subset_df), position=0, leave=True, desc="Progress"):
        image_path = os.path.join("/kaggle/input/abo-small-imgs/abo-images-small/abo-images-small", row['path'])

        if not os.path.exists(image_path):
            results.append("image_not_found")
            continue

        try:
            image = Image.open(image_path).convert("RGB")
            question = row["question"]
            inputs = processor(image, question, return_tensors="pt").to(device)
            out = model.generate(**inputs,**generation_args)
            decoded = processor.decode(out[0], skip_special_tokens=True)
            answer = re.sub(r"\s+", " ", decoded).strip()  # Normalize whitespace
        except Exception as e:
            answer = f"error: {str(e)}"

        results.append(answer)


    subset_df['blip_answer'] = results
    filename = f"blip_2_vqa_results_batch_{start_idx}_{end_idx}.csv"
    subset_df.to_csv(filename, index=False)
    print(f"Saved: {filename} with {len(subset_df)} rows.")



Progress: 100%|██████████| 2000/2000 [11:07<00:00,  3.00it/s]


Saved: blip_2_vqa_results_batch_0_2000.csv with 2000 rows.


Progress: 100%|██████████| 2000/2000 [11:08<00:00,  2.99it/s]


Saved: blip_2_vqa_results_batch_2000_4000.csv with 2000 rows.


Progress: 100%|██████████| 2000/2000 [11:08<00:00,  2.99it/s]


...

Saved: blip_2_vqa_results_batch_54000_56000.csv with 2000 rows.


Progress: 100%|██████████| 2000/2000 [10:54<00:00,  3.06it/s]


Saved: blip_2_vqa_results_batch_56000_58000.csv with 2000 rows.


Progress: 100%|██████████| 1847/1847 [10:05<00:00,  3.05it/s]

Saved: blip_2_vqa_results_batch_58000_59847.csv with 1847 rows.
